In [1]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

### input and output

In [19]:
###input
ymdb_output_final = '../../../Data/ymdb/ymdb_final_result.xlsx'
mnxreac_smile_atom_mapping_rules_file_path = '../../../Data/rules/MNXreaction_smiles_atommap_rules_lipid.csv'
target_smiles_path = '../../../Results/lipid/target_smiles_lipid.pickle'
target_model_smiles_path = '../../../Results/lipid/yeast_met_lipid.pickle'
yeast_model_path = '../../../Data/model/yeast-GEM-final.csv'
total_met_inchikey0_file = '../../../Data/total_met_inchikey0.pickle'
# uptake_met_path = '../../../Data/ymdb/ymdb_uptake.csv'
cut_off = 0.3
num = 'all'
num_process = 50
cut_off_path = f'../../../Results/lipid/top{num}_{cut_off}_re/'
cut_off_retrosynthesis_path = f'../../../Data_retrosynthesis/lipid/top{num}_{cut_off}_re/'

sce_gene_clean_ec = '../../../Data/Saccharomyces_cerevisiae_maxsep.csv'
DeepEC_path = '../../../Data/Sce_DeepECv2.txt'
sce_uniprot_path = '../../../Data/uniprotkb_organism_id_559292_2023_11_08.tsv'
yeast870_path =  '../../../Data/model/yeast-GEM.yml'
miss_met_id_file = '../../../Data/model/miss_met_id.json'
rxndb_to_model_with_target_success_path = cut_off_path + 'rxndb_to_model_with_target_topall_0.3_success.csv'
metabolites_info_to_GEM_path_not_lipid_path = '../../../Results/not_lipid/top50_0.3_re/metabolites_info_to_GEM_top50_0.3.csv'
###output
retrosys_smiles_calculate_similarity_file_path_all = f'../../../Data_retrosynthesis/lipid/retrosys_smiles_calculate_similarity_{cut_off}_re.json'
# retrosys_smiles_calculate_similarity_file_path_remain_all = f'../../../Data_retrosynthesis/lipid/retrosys_smiles_calculate_similarity_{cut_off}_remain_re.json'

retrosys_smiles_calculate_similarity_file_path = cut_off_retrosynthesis_path + f'retrosys_smiles_calculate_similarity_top{num}_{cut_off}.json'
retrosys_smiles_calculate_similarity_filter_file_path = cut_off_retrosynthesis_path + f'retrosys_smiles_calculate_similarity_filter_top{num}_{cut_off}.json'
rxndb_path = cut_off_retrosynthesis_path + f'RXNdb_top{num}_{cut_off}/'
failedrxn_path = cut_off_retrosynthesis_path + f'failedrxn_top{num}_{cut_off}.pickle' 
rxndb_all_path = cut_off_retrosynthesis_path + f'RXNdb_all_top{num}_{cut_off}.json'
# rxndb_all_path = cut_off_path + f'RXNdb_all_top{num}_{cut_off}_all.csv'
rxndb_drop_path = cut_off_retrosynthesis_path + f'RXNdb_drop_top{num}_{cut_off}/'
rxndb_met_max_score_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}.csv'
rxndb_met_max_score_uptake_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}_uptake.csv'
rxndb_to_model_add_ec_path = cut_off_path + f'rxndb_to_model_add_ec_top{num}_{cut_off}.csv'
rxndb_to_model_path = cut_off_path + f'rxndb_to_model_top{num}_{cut_off}.csv'
YMDB_success_met_smile_file = cut_off_path + f'YMDB_success_met_smile_top{num}_{cut_off}.pickle'
YMDB_success_met_smile_uptake_file = cut_off_path + f'YMDB_success_met_smile_uptake_top{num}_{cut_off}.pickle'
YMDB_fail_met_smile_file = cut_off_path + f'YMDB_fail_met_smile_top{num}_{cut_off}.pickle'
YMDB_fail_met_smile_uptake_file = cut_off_path + f'YMDB_fail_met_smile_uptake_top{num}_{cut_off}.pickle'
rxndb_GPR_to_model_path = cut_off_path + f'rxndb_GPR_to_model_top{num}_{cut_off}.csv'
rxndb_met_max_score_met_annotation_file = cut_off_path + f'rxndb_met_max_score_met_annotation_top{num}_{cut_off}.csv'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'
rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}.csv'
yeast8_reaction_in_rxndb_json = cut_off_path + f'yeast8_reaction_in_rxndb_top{num}_{cut_off}.json'




In [9]:
#check if the output folder exists
if not os.path.exists(cut_off_path):
    os.makedirs(cut_off_path)
if not os.path.exists(cut_off_retrosynthesis_path):
    os.makedirs(cut_off_retrosynthesis_path)
if not os.path.exists(rxndb_path):
    os.makedirs(rxndb_path)
if not os.path.exists(rxndb_drop_path):
    os.makedirs(rxndb_drop_path)

In [15]:
yeast_total_smiles = process_yeast_smiles(target_model_smiles_path)
# yeast_total_smiles = [neutralize_charge(x) for x in yeast_total_smiles]
yeast_total_smiles = [x for x in yeast_total_smiles if Chem.MolFromSmiles(x)]
retrorules = pd.read_csv(mnxreac_smile_atom_mapping_rules_file_path)
rules_product_smiles_lst = process_retrorules(retrorules,s='product_smiles')
rules_product_smarts_lst = process_retrorules(retrorules,s='ProductSMARTs')


total_smiles 19742
total_smiles 19741
total_smiles 19741
total_smiles 16523


13584it [00:01, 13265.17it/s]


6148


13584it [00:00, 14230.42it/s]

6682


In [11]:
retrosys_smiles_calculate_similarity = process_rules_with_multiprocessing(rules_product_smiles_lst, yeast_total_smiles,num_processes=num_process,cutoff=cut_off)

retrosys_smiles_calculate_similarity_pd = pd.DataFrame(retrosys_smiles_calculate_similarity)

100%|██████████| 6148/6148 [1:22:13<00:00,  1.25it/s] 


### get rxndb

In [12]:
retrorules = merge_smiles_similarity_rule(retrorules, retrosys_smiles_calculate_similarity_pd)


100%|██████████| 13584/13584 [00:43<00:00, 315.45it/s]


In [13]:
filter_smiles_muti(retrorules,retrosys_smiles_calculate_similarity_filter_file_path,num_process=num_process)
with open(retrosys_smiles_calculate_similarity_filter_file_path, 'r') as json_file:
    retrorules = json.load(json_file) 
retrorules = pd.DataFrame(retrorules)

100%|██████████| 50/50 [36:40<00:00, 44.01s/it]   


In [4]:
# files = os.listdir(rxndb_path)
# num_lst = [int(i.split('.')[0]) for i in files]
# for index,row in tqdm(retrorules.iterrows()):
#     if index  in num_lst:
#         retrorules.drop(index,inplace=True)
# retrorules.shape

13584it [00:26, 513.25it/s]


(4828, 21)

In [14]:
process_retrorules_and_save(rxndb_path,failedrxn_path,retrorules,heterologous_met_smiles=None,num_process=num_process)

100%|██████████| 13584/13584 [6:59:03<00:00,  1.85s/it]    


In [5]:
# process_retrorules_and_save(rxndb_path,failedrxn_path,retrorules,heterologous_met_smiles=None,num_process=num_process)


100%|██████████| 4828/4828 [3:09:38<00:00,  2.36s/it]  


### retrosynthesis result screen

In [3]:
#drop metabolites that are not in the ymdb or yeast8
drop_rxndb(rxndb_path, rxndb_drop_path, total_met_inchikey0_file, num_processes=num_process)
#statistic the number of reactions 
statistic_reaction_num(rxndb_path,rxndb_drop_path)

100%|██████████| 8697/8697 [00:10<00:00, 799.14it/s] 


rxndb: 179294368
rxndb_drop: 1364138


In [3]:
# #drop metabolites that are not in the ymdb or yeast8
# drop_rxndb(rxndb_path, rxndb_drop_path, total_met_inchikey0_file, num_processes=num_process)
# #statistic the number of reactions 
# statistic_reaction_num(rxndb_path,rxndb_drop_path)

100%|██████████| 8757/8757 [00:13<00:00, 663.11it/s] 


rxndb: 182790284
rxndb_drop: 1419580


In [4]:
#merge the rxndb
rxndb = merge_rxndb(rxndb_drop_path,rxndb_all_path)
rxndb_df = pd.DataFrame(rxndb).transpose()
print(rxndb_df.shape)
rxndb_df = rxndb_df.reset_index().rename(columns={'index':'NO'})


(1364138, 9)


In [5]:
# #merge the rxndb
# # rxndb = merge_rxndb(rxndb_path,rxndb_all_path)
# with open(rxndb_all_path,'r') as f:
#     rxndb = json.load(f)
# rxndb_df = pd.DataFrame(rxndb).transpose()
# print(rxndb_df.shape)
# rxndb_df = rxndb_df.reset_index().rename(columns={'index':'NO'})


In [5]:
#get all the metabolites in the rxndb
rxndb_all_smiles = parallel_process_rxn_smiles(rxndb_df, n_splits=100,num_process=num_process)

100%|██████████| 100/100 [00:10<00:00,  9.52it/s]


number of metabolites in RXNDB: 45560


In [6]:
#get all the smiles in the yeast8
yeast8_total_smiles = get_GEM_all_smiles(yeast_model_path)

In [7]:
#calculate the similarity of the metabolites

smile_max_score_pd = calculate_save_smiles_max_score(rxndb_all_smiles, yeast8_total_smiles,rxndb_met_max_score_file,num_processes=num_process)

100%|██████████| 45560/45560 [47:26<00:00, 16.01it/s]  


In [8]:
# smile_max_score_pd = pd.read_csv(rxndb_met_max_score_file)

In [9]:

smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_df, smile_max_score_pd, num_processes=num_process, num_iterations=9)
smiles_max_score_tmp_pd.to_csv(rxndb_met_max_score_file,index=None)





100%|██████████| 100/100 [03:36<00:00,  2.16s/it]


Iteration 1 - Current success count: 17082
Iteration 1 - Current success_rxndbid count: 377482


100%|██████████| 100/100 [03:55<00:00,  2.36s/it] 


Iteration 2 - Current success count: 27098
Iteration 2 - Current success_rxndbid count: 513059


100%|██████████| 100/100 [04:47<00:00,  2.88s/it] 


...

Iteration 7 - Current success count: 42302
Iteration 7 - Current success_rxndbid count: 1086622


100%|██████████| 100/100 [06:02<00:00,  3.63s/it] 


Iteration 8 - Current success count: 42305
Iteration 8 - Current success_rxndbid count: 1087887


100%|██████████| 100/100 [06:00<00:00,  3.61s/it] 


Iteration 9 - Current success count: 42305
Iteration 9 - Current success_rxndbid count: 1087887
final success 42305
final success_rxndbid 1087887


In [10]:
# smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_df, smiles_max_score_tmp_pd, num_processes=num_process, num_iterations=2)
# smiles_max_score_tmp_pd.to_csv(rxndb_met_max_score_file,index=None)

In [11]:
# smiles_max_score_tmp_pd = pd.read_csv(rxndb_met_max_score_file)

In [12]:

save_success_fail_target_smiles(target_smiles_path,smiles_success,YMDB_success_met_smile_file,YMDB_fail_met_smile_file)




success number: 12465
fail number: 1845


In [13]:
save_rxndb_to_model(rxndb_df,success_rxndbid,rxndb_to_model_path)


(1087887, 10)
(1087887, 10)
(1087887, 5)
(1087887, 5)


In [16]:
lipid_success = load_pickle(YMDB_success_met_smile_file)
lipid_success_inchikey0 = [smiles2inchikey0(x) for x in lipid_success]
yeast_total_smiles = [x for x in yeast_total_smiles if smiles2inchikey0(x) not in lipid_success_inchikey0]
len(yeast_total_smiles)

3453

In [5]:
###input


num_process = 40
cut_off_path = f'../../../Results/lipid/top{num}_{cut_off}_remain_re/'
mnxreac_smile_atom_mapping_rules_file_path = '../../Data/rules/MNXreaction_smiles_atommap_rules_filter_add_no_ec.csv'
cut_off_path_ = f'../../../Results/lipid/top{num}_{cut_off}_re/'
cut_off_retrosynthesis_path = f'../../../Data_retrosynthesis/lipid/top{num}_{cut_off}_remain_re/'


###output
retrosys_smiles_calculate_similarity_file_path_all = f'../../../Data_retrosynthesis/lipid/retrosys_smiles_calculate_similarity_{cut_off}_remain_re.json'

retrosys_smiles_calculate_similarity_file_path = cut_off_retrosynthesis_path + f'retrosys_smiles_calculate_similarity_top{num}_{cut_off}.json'
retrosys_smiles_calculate_similarity_filter_file_path = cut_off_retrosynthesis_path + f'retrosys_smiles_calculate_similarity_filter_top{num}_{cut_off}.json'
rxndb_path = cut_off_retrosynthesis_path + f'RXNdb_top{num}_{cut_off}/'
failedrxn_path = cut_off_retrosynthesis_path + f'failedrxn_top{num}_{cut_off}.pickle' 
rxndb_all_path = cut_off_retrosynthesis_path + f'RXNdb_all_top{num}_{cut_off}.json'
# rxndb_all_path = cut_off_path + f'RXNdb_all_top{num}_{cut_off}_all.csv'
rxndb_drop_path = cut_off_retrosynthesis_path + f'RXNdb_drop_top{num}_{cut_off}/'
rxndb_met_max_score_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}.csv'
# rxndb_met_max_score_uptake_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}_uptake.csv'
rxndb_to_model_path_remain  = cut_off_path + f'rxndb_to_model_top{num}_{cut_off}_remain.csv'
rxndb_to_model_path = cut_off_path_ + f'rxndb_to_model_top{num}_{cut_off}.csv'
# rxndb_to_model_total_path = '../../../Results/lipid/' + f'rxndb_to_model_total_top{num}_{cut_off}_total.csv'
YMDB_success_met_smile_file_remian = cut_off_path + f'YMDB_success_met_smile_top{num}_{cut_off}_remain.pickle'
# YMDB_success_met_smile_file = cut_off_path + f'YMDB_success_met_smile_top{num}_{cut_off}.pickle'
YMDB_fail_met_smile_file_remain = cut_off_path + f'YMDB_fail_met_smile_top{num}_{cut_off}_remain.pickle'
YMDB_fail_met_smile_file = cut_off_path_ + f'YMDB_fail_met_smile_top{num}_{cut_off}.pickle'




In [18]:
if not os.path.exists(cut_off_path):
    os.makedirs(cut_off_path)
if not os.path.exists(cut_off_retrosynthesis_path):
    os.makedirs(cut_off_retrosynthesis_path)
if not os.path.exists(rxndb_path):
    os.makedirs(rxndb_path)
if not os.path.exists(rxndb_drop_path):
    os.makedirs(rxndb_drop_path)

In [19]:
retrorules = pd.read_csv(mnxreac_smile_atom_mapping_rules_file_path)
rules_product_smiles_lst = process_retrorules(retrorules,s='product_smiles')
rules_product_smarts_lst = process_retrorules(retrorules,s='ProductSMARTs')

29164it [00:03, 9565.70it/s] 


11134


29164it [00:04, 7197.87it/s] 

16152


In [20]:
retrosys_smiles_calculate_similarity = process_rules_with_multiprocessing(rules_product_smiles_lst, yeast_total_smiles,num_processes=num_process,cutoff=cut_off)

retrosys_smiles_calculate_similarity_pd = pd.DataFrame(retrosys_smiles_calculate_similarity)
retrosys_smiles_calculate_similarity_pd.to_json(retrosys_smiles_calculate_similarity_file_path_all)

# choose top num similar metabolites
# retrosys_smiles_calculate_similarity_pd = pd.read_json(retrosys_smiles_calculate_similarity_file_path_all)
# retrosys_smiles_calculate_similarity_pd = get_most_similar_smiles(retrosys_smiles_calculate_similarity_pd,num)
# retrosys_smiles_calculate_similarity_pd.to_json(retrosys_smiles_calculate_similarity_file_path)

100%|██████████| 11134/11134 [32:02<00:00,  5.79it/s] 


In [21]:
retrorules = merge_smiles_similarity_rule(retrorules, retrosys_smiles_calculate_similarity_pd)
filter_smiles_muti(retrorules,retrosys_smiles_calculate_similarity_filter_file_path,num_process=num_process)
with open(retrosys_smiles_calculate_similarity_filter_file_path, 'r') as json_file:
    retrorules = json.load(json_file) 
retrorules = pd.DataFrame(retrorules)

100%|██████████| 40/40 [10:39<00:00, 15.99s/it]  


In [22]:
process_retrorules_and_save(rxndb_path,failedrxn_path,retrorules,heterologous_met_smiles=None,num_process=25)


100%|██████████| 29164/29164 [29:16:25<00:00,  3.61s/it]     


In [6]:
# files = os.listdir(rxndb_path)
# num_lst = [int(i.split('.')[0]) for i in files]
# for index,row in tqdm(retrorules.iterrows()):
#     if index  in num_lst:
#         retrorules.drop(index,inplace=True)
# retrorules.shape

29164it [01:25, 342.41it/s] 


(18464, 20)

In [7]:
# process_retrorules_and_save(rxndb_path,failedrxn_path,retrorules,heterologous_met_smiles=None,num_process=num_process)


100%|██████████| 18464/18464 [13:52:43<00:00,  2.71s/it]    


In [23]:
#drop metabolites that are not in the ymdb or yeast8
drop_rxndb(rxndb_path, rxndb_drop_path, total_met_inchikey0_file, num_processes=30)
#statistic the number of reactions 
statistic_reaction_num(rxndb_path,rxndb_drop_path)

100%|██████████| 17238/17238 [00:09<00:00, 1758.90it/s]


rxndb: 885993664
rxndb_drop: 1813688


In [24]:
#merge the rxndb
rxndb = merge_rxndb(rxndb_drop_path,rxndb_all_path)
rxndb_df = pd.DataFrame(rxndb).transpose()
print(rxndb_df.shape)

(1813688, 9)


In [ ]:
# with open(rxndb_all_path,'r') as f:
#     rxndb = json.load(f)
# rxndb_df = pd.DataFrame(rxndb).transpose()
# print(rxndb_df.shape)

(1882712, 9)


In [25]:
rxndb_df = rxndb_df.reset_index().rename(columns={'index':'NO'})


In [26]:
rxndb_all_smiles = parallel_process_rxn_smiles(rxndb_df, n_splits=100,num_process=num_process)


100%|██████████| 100/100 [00:14<00:00,  6.74it/s]


number of metabolites in RXNDB: 12835


In [27]:
#get all the smiles in the yeast8
yeast8_total_smiles = get_GEM_all_smiles(yeast_model_path)

# # consider uptake metabolites
# uptake = pd.read_csv(uptake_met_path)
# uptake_smiles = uptake['SMILES'].values
# uptake_smiles = [normalize_smiles(i) for i in uptake_smiles]
# yeast8_total_smiles = yeast8_total_smiles + uptake_smiles

In [28]:
smile_max_score_pd = calculate_save_smiles_max_score(rxndb_all_smiles, yeast8_total_smiles,rxndb_met_max_score_file,num_processes=num_process)


100%|██████████| 12835/12835 [27:45<00:00,  7.71it/s] 


In [6]:
rxndb_df = pd.read_csv(rxndb_to_model_path_remain)
smiles_max_score_tmp_pd = pd.read_csv(rxndb_met_max_score_file)

In [7]:
# smiles_max_score_tmp_pd = pd.read_csv(rxndb_met_max_score_uptake_file)

smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_df, smiles_max_score_tmp_pd, num_processes=50, num_iterations=1)
# smiles_max_score_tmp_pd.to_csv(rxndb_met_max_score_file,index=None)





100%|██████████| 100/100 [03:06<00:00,  1.86s/it]


Iteration 1 - Current success count: 10784
Iteration 1 - Current success_rxndbid count: 1655549
final success 10784
final success_rxndbid 1655549


In [33]:
# smiles_max_score_tmp_pd = pd.read_csv(rxndb_met_max_score_file)

# smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_df, smiles_max_score_tmp_pd, num_processes=num_process, num_iterations=1)
smiles_max_score_tmp_pd.to_csv(rxndb_met_max_score_file,index=None)





In [8]:
save_success_fail_target_smiles(YMDB_fail_met_smile_file,smiles_success,YMDB_success_met_smile_file_remian,YMDB_fail_met_smile_file_remain)


success number: 1045
fail number: 800


In [35]:
save_rxndb_to_model(rxndb_df,success_rxndbid,rxndb_to_model_path_remain)


(1655549, 10)
(1655549, 10)
(1655549, 5)
(1655549, 5)


### RXNDB gene annotation

In [4]:
#run find_no_ec_reaction_lipid_re.ipynb to get the rxndb_to_model_with_target_topall_0.3_success.csv file
data_add_no_ec = pd.read_csv(rxndb_to_model_with_target_success_path)
data_no_ec = data_add_no_ec[data_add_no_ec['EC number'].isnull()]
data_ec = data_add_no_ec[~data_add_no_ec['EC number'].isnull()]



In [10]:
data_no_ec.shape

(147323, 5)

In [5]:
data = pd.read_csv(rxndb_to_model_path)
print('data:', data.shape)

print('Unique NO values in data before merge:', data['NO'].nunique())

data = pd.concat([data, data_ec])


data = data.drop(columns=['NO'])

data = data.reset_index(drop=True)
data.index.name = 'NO'
data = data.reset_index()

print( data['NO'].nunique())
data.to_csv(rxndb_to_model_add_ec_path,index=None)


data: (1087887, 5)
Unique NO values in data before merge: 1087887
1390450


In [6]:
model = cobra.io.load_yaml_model(yeast870_path)

model_genes = model.genes

model_genes = [x.id for x in model_genes]

### CLEAN

In [7]:
clean_gene2ec_dict = get_gene2ec_dict_clean(sce_gene_clean_ec)

### DeepECv2

In [8]:
DeepEC_gene2ec_dict = get_gene2ec_dict_DeepEC(DeepEC_path)

In [9]:
clean_deepec_gene2ec_dict = {}
for k,v in clean_gene2ec_dict.items():
    if k in DeepEC_gene2ec_dict or k in model_genes:
        clean_deepec_gene2ec_dict[k] = v
clean_deepec_ec2gene_dict = get_ec2gene_dict_clean(clean_deepec_gene2ec_dict)
print(len(clean_deepec_ec2gene_dict))

161


In [11]:
rxndb_with_GPR = rxndb_gene_annotation(rxndb_to_model_add_ec_path,clean_deepec_ec2gene_dict)
rxndb_with_GPR = rxndb_with_GPR[rxndb_with_GPR['GENE'].apply(lambda x: len(x) > 0)]
data_no_ec['GENE'] = None
data_no_ec['GENE'] = data_no_ec['GENE'].apply(lambda x: [])
data_no_ec['EC number'] = data_no_ec['EC number'].apply(lambda x: [])
print('rxndb with GPR:',rxndb_with_GPR.shape)


rxndb all ec num 124


/tmp/ipykernel_3234758/3407858261.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_ec['GENE'] = None


rxndb with GPR: (1387919, 6)


/tmp/ipykernel_3234758/3407858261.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_ec['GENE'] = data_no_ec['GENE'].apply(lambda x: [])
/tmp/ipykernel_3234758/3407858261.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_ec['EC number'] = data_no_ec['EC number'].apply(lambda x: [])


In [12]:
print('rxndb_with_GPR:',rxndb_with_GPR.shape)

rxndb_with_GPR = pd.concat([rxndb_with_GPR,data_no_ec])
rxndb_with_GPR = rxndb_with_GPR.drop(columns=['NO'])
rxndb_with_GPR = rxndb_with_GPR.reset_index(drop=True)
rxndb_with_GPR.index.name = 'NO'
rxndb_with_GPR = rxndb_with_GPR.reset_index()

print( rxndb_with_GPR['NO'].nunique())
# rxndb_with_GPR.to_csv(rxndb_GPR_to_model_path,index=None)

rxndb_with_GPR: (1387919, 6)
1535242


In [13]:
rxndb_with_GPR['NO'] = rxndb_with_GPR['NO'].apply(lambda x: f'rxnl{x}')
rxndb_with_GPR.head()


,NO,EC number,templateID,rxn_smiles_basic,rxn_smiles_final,GENE
0,rxnl0,[1.1.1],MNXR100471_reverse,O=C([O-])C[C@@H](O)C(=O)[O-]>>O=C([O-])CC(=O)C...,O=C([O-])C[C@@H](O)C(=O)[O-].NC(=O)c1ccc[n+]([...,"[YDR536W, YLR174W, YHR183W, YJL214W, YJR096W, ..."
1,rxnl1,[1.1.1],MNXR100471_reverse,C[C@H](O)CC(=O)[O-]>>CC(=O)CC(=O)[O-],C[C@H](O)CC(=O)[O-].NC(=O)c1ccc[n+]([C@@H]2O[C...,"[YDR536W, YLR174W, YHR183W, YJL214W, YJR096W, ..."
2,rxnl2,[1.1.1],MNXR100471_reverse,CC(C)C(C(=O)[O-])[C@@H](O)C(=O)[O-]>>CC(C)C(C(...,CC(C)C(C(=O)[O-])[C@@H](O)C(=O)[O-].NC(=O)c1cc...,"[YDR536W, YLR174W, YHR183W, YJL214W, YJR096W, ..."
3,rxnl3,[1.1.1],MNXR100471_reverse,C[C@H](O)C([NH3+])C(=O)[O-]>>CC(=O)C([NH3+])C(...,C[C@H](O)C([NH3+])C(=O)[O-].NC(=O)c1ccc[n+]([C...,"[YDR536W, YLR174W, YHR183W, YJL214W, YJR096W, ..."
4,rxnl4,"[3.2.1, 3.6.1]",MNXR132170,Nc1nc2c(ncn2C2OC(COP(=O)([O-])OP(=O)([O-])[O-]...,Nc1nc2c(ncn2C2OC(COP(=O)([O-])OP(=O)([O-])[O-]...,"[YPR026W, YJL216C, YOR190W, YIL162W, YKR005C, ..."


In [14]:
rxndb_with_GPR.to_csv(rxndb_GPR_to_model_path,index=None)


### rxndb metabolite annotation

In [15]:
#rxndb all metabolites
rxndb_all_smile = get_all_met_smile(rxndb_GPR_to_model_path)


100%|██████████| 1535242/1535242 [15:56<00:00, 1604.47it/s] 


45794


In [16]:
#metabolite comparison
yeast8_total_smiles = get_GEM_all_smiles(yeast_model_path)
smile_max_score = calculate_smile_max_score_annotation(rxndb_all_smile,yeast8_total_smiles,num_process=num_process)
smile_max_score_pd = pd.DataFrame(smile_max_score)
smile_max_score_pd.to_csv(rxndb_met_max_score_met_annotation_file,index=None)


100%|██████████| 45794/45794 [36:31<00:00, 20.89it/s]  


In [17]:
#assign ID
yeast8_id_smiles_mapping = get_yeast8_id_smiles_mapping(yeast_model_path)
smile_max_score_pd_with_ID = assign_ID_to_RXNDB_met(smile_max_score_pd,yeast8_id_smiles_mapping,s = 'lipid')
metabolites_info_to_GEM = get_new_met_smile_list(smile_max_score_pd_with_ID)
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)



/home/haohao/underground/Underground_workflow/Code/retrosynthesis/lipid_retrosynthesis/../common.py:1396: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yeast_id_mapping['smiles'] = yeast_id_mapping['smiles'].apply(normalize_smiles)
/home/haohao/underground/Underground_workflow/Code/retrosynthesis/lipid_retrosynthesis/../common.py:1397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yeast_id_mapping['inchikey0'] = yeast_id_mapping['smiles'].apply(smiles2inchikey0)
100%|██████████| 45794/45794 [00:44<00:00, 10

In [20]:
metabolites_info_to_GEM_path_not_lipid = pd.read_csv(metabolites_info_to_GEM_path_not_lipid_path)
metabolites_info_to_GEM_path_not_lipid['inchikey0'] = metabolites_info_to_GEM_path_not_lipid['new_met_smiles'].apply(smiles2inchikey0)
for i in tqdm(range(metabolites_info_to_GEM.shape[0]),total=metabolites_info_to_GEM.shape[0]):
    if 'sl_' not in metabolites_info_to_GEM['ID'][i]:
        continue
    if smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i]) in metabolites_info_to_GEM_path_not_lipid['inchikey0'].values:
        metabolites_info_to_GEM['ID'][i] = metabolites_info_to_GEM_path_not_lipid[metabolites_info_to_GEM_path_not_lipid['inchikey0'] == smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i])]['ID'].values[0]
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)

  0%|          | 0/45794 [00:00<?, ?it/s]/tmp/ipykernel_3234758/1497864285.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metabolites_info_to_GEM['ID'][i] = metabolites_info_to_GEM_path_not_lipid[metabolites_info_to_GEM_path_not_lipid['inchikey0'] == smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i])]['ID'].values[0]
/tmp/ipykernel_3234758/1497864285.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metabolites_info_to_GEM['ID'][i] = metabolites_info_to_GEM_path_not_lipid[metabolites_info_to_GEM_path_not_lipid['inchikey0'] == smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i])]['ID'].value

### Generate equation according to metabolite ID

In [6]:
smiles_to_id_mapping = get_smiles_to_id_mapping(metabolites_info_to_GEM_path)
print(len(smiles_to_id_mapping))

4166


In [7]:
rxndb_total_info_to_model = get_rxndb_to_model_total_info(rxndb_GPR_to_model_path,smiles_to_id_mapping)
rxndb_total_info_to_model = rxndb_total_info_to_model[['NO','EC_number','templateID','rxn_smiles_basic','rxn_smiles_final','GPR','equation_dict','equation']]
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)
print(rxndb_total_info_to_model.shape)

(53819, 8)


### check the formula of the model metabolites in the predicted reaction

In [8]:
metabolites_info_to_GEM,rxndb_total_info_to_model = check_met_and_reaction_formula(yeast_model_path,metabolites_info_to_GEM_path,rxndb_total_info_to_model_path)
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)
print(rxndb_total_info_to_model.shape)

formula error_met set()


0it [00:00, ?it/s]


(53819, 8)


### remove duplicate reactions and existing reactions

In [9]:
same_rxndb_reaction = get_same_rxndb_reaction_mapping(rxndb_total_info_to_model_path)
rxndb_total_info_to_model = get_no_same_rxndb_reaction_parallelize(rxndb_total_info_to_model_path,get_no_same_rxndb_reaction,same_rxndb_reaction,n_cores=num_process)
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)
print(rxndb_total_info_to_model.shape)

(53819, 8)


100%|██████████| 53819/53819 [00:02<00:00, 26145.44it/s]


(6293, 8)


In [10]:
met_info = pd.read_csv(metabolites_info_to_GEM_path)
rxndb_to_model_total_info = pd.read_csv(rxndb_total_info_to_model_path)


### Find out the reactions that already exist in the model

In [11]:
with open(miss_met_id_file) as f:
    miss_met_id = json.load(f) 
model = cobra.io.load_yaml_model(yeast870_path)


### yeast8_smiles_list, ID with smiles

In [12]:
model_excel, metID_with_smiles, yeast8_smiles_lst = process_model(yeast_model_path, miss_met_id)


2124
['s_0006', 's_0007', 's_0008', 's_0009', 's_0010', 's_0011', 's_0012', 's_0013', 's_0014', 's_0015', 's_0016', 's_0017', 's_0018', 's_0019', 's_0020', 's_0021', 's_0022', 's_0023', 's_0024', 's_0025', 's_0026', 's_0027', 's_0028', 's_0029', 's_0030', 's_0031', 's_0032', 's_0033', 's_0034', 's_0035', 's_0036', 's_0037', 's_0038', 's_0039', 's_0042', 's_0045', 's_0048', 's_0051', 's_0054', 's_0056', 's_0058', 's_0060', 's_0061', 's_0062', 's_0063', 's_0064', 's_0065', 's_0066', 's_0067', 's_0068', 's_0069', 's_0075', 's_0076', 's_0077', 's_0078', 's_0079', 's_0080', 's_0086', 's_0087', 's_0116', 's_0117', 's_0118', 's_0119', 's_0120', 's_0121', 's_0122', 's_0123', 's_0124', 's_0125', 's_0126', 's_0127', 's_0129', 's_0131', 's_0132', 's_0133', 's_0134', 's_0135', 's_0136', 's_0137', 's_0138', 's_0139', 's_0140', 's_0141', 's_0142', 's_0143', 's_0144', 's_0145', 's_0146', 's_0147', 's_0148', 's_0149', 's_0150', 's_0152', 's_0153', 's_0154', 's_0155', 's_0156', 's_0157', 's_0158', 's_0

In [13]:
rxndb_met_with_s_smiles_lst = get_rxndb_met_with_s_smiles_lst(rxndb_total_info_to_model_path,model_excel,miss_met_id)
reaID_with_smiles_dict = get_reaID_with_smiles_dict(model, model_excel, miss_met_id,rxndb_met_with_s_smiles_lst,metID_with_smiles)


(6293, 8)


100%|██████████| 6293/6293 [00:05<00:00, 1065.63it/s]


801
['CC(C)=CCCC(C)C1CCC2C3=C(CCC21C)C1(C)CCC(O)CC1CC3', 'CC(C)=CCCC(C)C1CCC2C3=C(CCC21C)C1(C)CCC(=O)CC1CC3', 'CC(C)=CCCC(C)C1CCC2C3=C(CCC21C)C1(C)CCC(O)C(C)C1CC3', 'CC(C)=CCCC(C)C1CCC2C3=C(CCC21C)C1(C)CCC(=O)C(C)C1CC3', 'CC(N)C(=O)O', 'O=CCO', 'CC(=O)C(=O)[O-]', 'NCCO', 'C=O', 'NC(CCC(=O)NC(CS)C(=O)NCC(=O)O)C(=O)O']
2702
1111
{'r_0003': ['CC(O)C(C)O', 'CC(=O)C(C)O'], 'r_0007': ['NC(=O)c1ncn(C2OC(COP(=O)([O-])[O-])C(O)C2O)c1N=CNC1OC(COP(=O)([O-])[O-])C(O)C1O', 'NC(=O)c1ncn(C2OC(COP(=O)([O-])[O-])C(O)C2O)c1NC=NCC(=O)C(O)C(O)COP(=O)([O-])[O-]'], 'r_0012': ['O=C(O)C1CCC=N1', 'NC(CCC(=O)O)C(=O)O'], 'r_0013': ['O=C[O-]', 'CSCCC(=O)C(=O)COP(=O)(O)O', 'CSCCC(=O)C(=O)O'], 'r_0014': ['Nc1c(NCC(O)C(O)C(O)COP(=O)(O)O)[nH]c(=O)[nH]c1=O', 'Nc1nc(NCC(O)C(O)C(O)COP(=O)(O)O)c(N)c(=O)[nH]1'], 'r_0015': ['Nc1nc(NC2OC(COP(=O)(O)O)C(O)C2O)c(N)c(=O)[nH]1', 'Nc1nc(NCC(O)C(O)C(O)COP(=O)(O)O)c(N)c(=O)[nH]1'], 'r_0016': ['CCC(=O)C(=O)[O-]', 'O=C=O', 'CC(=O)C(=O)[O-]', 'CCC(O)(C(C)=O)C(=O)[O-]'], 'r_0018': ['O=

In [14]:
yeast8_reaction_in_rxndb = get_yeast8_reaction_in_rxndb_parallel(rxndb_total_info_to_model_path,reaID_with_smiles_dict,model_excel,miss_met_id,num_process=num_process)


(6293, 8)
(4108, 8)


In [15]:
with open(yeast8_reaction_in_rxndb_json, 'w') as json_file:
    json.dump(yeast8_reaction_in_rxndb, json_file,indent=4)

In [ ]:
rxndb_to_model_total_info['equation_dict'] = rxndb_to_model_total_info['equation_dict'].apply(lambda x:literal_eval(x))
print(rxndb_to_model_total_info.shape)
# rxndb_to_model_total_info['equation_dict'] = rxndb_to_model_total_info['equation_dict'].apply(lambda x:{k:v for k, v in x.items() if k not in miss_met_id})
rxndb_to_model_total_info = rxndb_to_model_total_info[rxndb_to_model_total_info['equation_dict'].apply(lambda x : all('s_' in k for k, v in x.items()))]
print(rxndb_to_model_total_info.shape)
rxndb_to_model_total_info.to_csv(rxndb_total_info_to_model_path,index=None)

In [16]:
get_reaction_recovery_ratio(yeast8_reaction_in_rxndb)
#cut off 0.3
#top20,0.791
#top30,0.795
#top40,0.8
#top50,0.8
#top60,0.806
#top70,0.826
#top80,0.827

0.8037803780378038


In [28]:
# yeast8_reaction_in_rxndb = {k:v for k,v in yeast8_reaction_in_rxndb.items() if len(v)>0}



### Check the accuracy of GPR

In [ ]:
#Gene2ec was obtained according to uniprot
# sce_gene2ec_dict = get_sce_gene2ec_dict(sce_uniprot_path)


In [ ]:
# check_gpr_accuracy(yeast8_reaction_in_rxndb,model,sce_gene2ec_dict,rxndb_to_model_total_info)
#top20,0.849
#top30,0.852
#top40,0.851
#top50,0.854
#top60,0.861
#top70,0.864
#top80,0.871